# Building a Euribor Curve using non-parametric calibration with smoothing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from timeit import default_timer as timer

In [ ]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.rates.ibor_single_curve_smoothing_calibrator import IborSingleCurveSmoothingCalibrator

In [ ]:
valuation_date = Date(6, 10, 2001)
cal = CalendarTypes.UNITED_KINGDOM

### Instruments

In [ ]:
depoDCCType = DayCountTypes.ACT_360
depos = []
spot_days = 2
settlement_date = valuation_date.add_weekdays(spot_days)
depo = IborDeposit(settlement_date, "3M", 4.2/100.0, depoDCCType, cal_type=cal)
depos.append(depo)

fraDCCType = DayCountTypes.ACT_360
fras = []
fra = IborFRA(settlement_date.add_tenor("3M"), "3M", 4.20/100.0, fraDCCType, cal_type=cal)
fras.append(fra)

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.THIRTY_E_360_ISDA
fixed_freqType = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settlement_date, "1Y", swapType, 4.20/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "2Y", swapType, 4.30/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "3Y", swapType, 4.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "5Y", swapType, 5.40/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "7Y", swapType, 5.70/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "10Y", swapType, 6.00/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "12Y", swapType, 6.10/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "15Y", swapType, 5.90/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "20Y", swapType, 5.60/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)
swap = IborSwap(settlement_date, "25Y", swapType, 5.55/100.0, fixed_freqType, fixedDCCType, cal_type=cal)
swaps.append(swap)

Build curves for different levels of the smoothness parameter

In [ ]:
check_refit = True
iborCurves = {}
full_report = None
optional_interp_params = {'sigma' : 0.01} # only relevant for interp_type == InterpTypes.TENSION_ZERO_RATES

# Choose your interpolation. InterpTypes.FLAT_FWD_RATES is the fastest but others are interesting too
interp_type = InterpTypes.FLAT_FWD_RATES
# interp_type = InterpTypes.NATCUBIC_ZERO_RATES

# Create but do not build the initial curve
do_build = False
init_curve = IborSingleCurve(valuation_date, depos, fras, swaps,
                                interp_type, check_refit=False, do_build=do_build, **optional_interp_params)
calibrator = IborSingleCurveSmoothingCalibrator(init_curve)

smooth_params = [0.01, 0.1, 0.2]
for smoothness  in smooth_params:
    start = timer()

    # Here we go, the main fitting call
    iborCurve, fit_report = calibrator.fit(smoothness=smoothness, report_progress = False)
    end = timer()
    print(f'smth = {smoothness} fit took {end-start:.2f} sec')
    iborCurves[smoothness] = iborCurve

    if full_report is None:
        full_report = fit_report
    else:
        full_report = full_report.append(fit_report, ignore_index = True)


### Look at the fit reports

In [ ]:
# Print DataFrames in full
pd.set_option('display.max_rows', None,
                    'display.max_columns', None,
                    'display.float_format', lambda x:f'{x:.4f}'
                    )

In [ ]:
# examine the last fit_report
fit_report.head()

In [ ]:
# combine fit reports for different smothness parameters into one for ease of comparison
fr2 = full_report[['tgt_label','value_in_bps', 'smoothness']].set_index(['tgt_label', 'smoothness']).unstack(level=1).droplevel(0,axis=1)
fr2 = fr2.join(fit_report[['tgt_label','type','start_date',	'maturity_date']].set_index('tgt_label'))
fr2

### Curve Shape Analysis

First check instantaneous (ON) forward rates and see how they are affected by the smoothness parameter

In [ ]:
years = np.linspace(1./365, 25, 25*365)
plotDates = settlement_date.add_years(years)

%matplotlib widget
plt.figure(figsize=(8,5))
for smoothness  in smooth_params:

    iborCurve = iborCurves[smoothness]
    iborCurveFwdRate = iborCurve.fwd(plotDates)   
    plt.plot(years, iborCurveFwdRate*100.0, label=f"smth={smoothness}")

plt.title(interp_type)
plt.xlabel("Years")
plt.ylabel("Rate (%)")
plt.title('ON fwd rates')
plt.legend(loc = 'best')
plt.show()

Observe smoothness at work for zero rates (yields)

In [ ]:
years = np.linspace(1./365, 25, 25*365)
plotDates = settlement_date.add_years(years)

%matplotlib widget
plt.figure(figsize=(8,5))
for smoothness  in smooth_params:

    iborCurve = iborCurves[smoothness]
    iborCurveZeroRate = iborCurve.zero_rate(plotDates)    
    plt.plot(years, iborCurveZeroRate*100.0, label=f"smth={smoothness}")

plt.title(interp_type)
plt.xlabel("Years")
plt.ylabel("Rate (%)")
plt.title('Yields (zero rates)')
plt.legend(loc = 'best')
plt.show()


Compare output swap rates vs targets

In [ ]:
%matplotlib widget

mat_years = np.linspace(1,25,25,endpoint = True)
mat_dates = settlement_date.add_years(mat_years)

tgt_dates = [s.fixed_leg.maturity_dt for s in swaps]
tgt_years = [(d - valuation_date)/g_days_in_year for d in tgt_dates]
tgt_rates = np.array([s.fixed_leg.cpn for s in swaps])

plt.figure(figsize=(8,5))


plt.plot(tgt_years, tgt_rates*100.0, 'o', label=f"swap rate targets")

for smoothness  in smooth_params:

    iborCurve = iborCurves[smoothness]
    iborCurveSwapRates = iborCurve.swap_rate(swap.effective_dt, mat_dates, 
        swap.fixed_leg.freq_type, swap.fixed_leg.dc_type)    
    plt.plot(mat_years, iborCurveSwapRates*100.0, '.', label=f"smth={smoothness}")

plt.title(interp_type)
plt.xlabel("Years")
plt.ylabel("Rate (%)")
plt.title('swap rates')
plt.legend(loc = 'best')
plt.show()